In [140]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D, GRU
from keras.layers import Conv1D, MaxPooling1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.layers import Bidirectional
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
## Loading Data - labelled data:

data_1 = pd.ExcelFile('R1_RNN_Tranzact_Rohan_1_Regrouped_with_supplier.xlsx')
tranzact_data_1 = data_1.parse(0)

tranzact_data_1.tail()

,supplier_item_id,to_company_name,supplier_item_id.1,supplier_item_id.2,product,supplier,Done,Atul Sugg.,sub_type,material,process,grade,spec,brand
13552,86390,CESARE BONETTI INTERNATIONAL PVT. LTD.,86390,"YOKE SLEEVE;8"" # 150;GTV; B150 C62300",YOKE SLEEVE,CESARE BONETTI INDIA PVT. LTD.,278.0,NaN,NaN,NaN,NaN,C62300,NaN,NaN
13553,89723,CESARE BONETTI INTERNATIONAL PVT. LTD.,89723,"YOKE SLEEVE 1½"" #2700 BLY;B150 C62300",YOKE SLEEVE,CESARE BONETTI INDIA PVT. LTD.,278.0,NaN,NaN,NaN,NaN,C62300,NaN,NaN
13554,89995,CESARE BONETTI INTERNATIONAL PVT. LTD.,89995,"YOKE SLEEVE;4"" #150 GTV; B150 C62300",YOKE SLEEVE,CESARE BONETTI INDIA PVT. LTD.,278.0,NaN,NaN,NaN,NaN,C62300,NaN,NaN
13555,117819,WAAREE INDUSTRIES PVT. LTD.,117819,"YOKE SLEEVE;8"" # 150;GTV; A439 D2",cicasting,Waaree Industries Pvt.Ltd.,278.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13556,169292,CESARE BONETTI INTERNATIONAL PVT. LTD.,169292,"YOKE SLEEVE; 1"";#1500;CBD;A582 T416",YOKE SLEEVE,CESARE BONETTI INDIA PVT. LTD.,278.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
## Loading Data - un-labelled data:

data_2 = pd.ExcelFile('14Mar_Gorky.xlsx')
tranzact_data_new_1 = data_2.parse(0)

tranzact_data_new_1.tail()

,buyer_item_id,buyer_itemid,buyer_item_name,from_company_id,from_company_name,to_company_id,to_company_name
17763,289052,PLYBAG_10X15,POLYBAG_10X15 INCH_PP,13872,R-Tech Products & Packaging Pvt Ltd,16730,GLORY PACK INDUSTRIES
17764,289639,HANDLENUTM12X5MMXX,"HANDLE NUT 1"" #600,HTB,A194 Gr8",842,CESARE BONETTI INDIA PVT. LTD.,7434,Shah Brothers
17765,48512,RDBR50 (SS431),ROUND BAR OD50 ASTM A276 TYPE 431,114,Entech Controls,921,ALPESH METALS
17766,57696,PGX618101601202200,"PTV 6"" #300 RF BVE WCB HW IBR",7506,CESARE BONETTI INTERNATIONAL PVT. LTD.,842,CESARE BONETTI INDIA PVT. LTD.
17767,274954,PPBOX_ST5,PP_MONOBOX_ST5,13872,R-Tech Products & Packaging Pvt Ltd,15527,Shree Arun Packaging Co. Pvt. Ltd.


In [8]:
## Picking up required info from labelled data:

col = ['to_company_name', 'supplier_item_id.2', 'product', 'supplier']
tranzact_data_2 = tranzact_data_1[col]
tranzact_data_2.columns = ['OEM', 'item', 'product', 'supplier']

# Creating separate dataframe for Cesare Bonetti International Pvt. Ltd. since these are unlabelled :

tranzact_data_cesare_international = tranzact_data_2.loc[tranzact_data_2['OEM'] \
                                                               == 'CESARE BONETTI INTERNATIONAL PVT. LTD.']

tranzact_data_2_revised = tranzact_data_2.drop(tranzact_data_cesare_international.index)

tranzact_data_cesare_international = tranzact_data_cesare_international.reset_index(drop=True)
tranzact_data_2_revised = tranzact_data_2_revised.reset_index(drop=True)

## Picking up required info from un-labelled data:

col = ['from_company_name', 'buyer_item_name', 'to_company_name']
tranzact_data_new_2 = tranzact_data_new_1[col]
tranzact_data_new_2.columns = ['OEM', 'item', 'supplier']

## Merging two data frames :

tranzact_data_combined = pd.concat([tranzact_data_2_revised, tranzact_data_new_2], axis=0, ignore_index=True)



## Removing duplicate (item + supplier) combinations :

tranzact_data_combined['find_duplicate'] = tranzact_data_combined['item'].map(str) + \
                                            tranzact_data_combined['supplier'].map(str)

tranzact_data_combined.drop_duplicates(subset='find_duplicate', keep = 'first', inplace = True)
tranzact_data_combined = tranzact_data_combined.reset_index(drop=True)
        
print(len(tranzact_data_combined))

tranzact_data_combined.tail()

29063


,OEM,item,product,supplier,find_duplicate
29058,R-Tech Products & Packaging Pvt Ltd,POLYBAG_10X15 INCH_PP,NaN,GLORY PACK INDUSTRIES,POLYBAG_10X15 INCH_PPGLORY PACK INDUSTRIES
29059,CESARE BONETTI INDIA PVT. LTD.,"HANDLE NUT 1"" #600,HTB,A194 Gr8",NaN,Shah Brothers,"HANDLE NUT 1"" #600,HTB,A194 Gr8Shah Brothers"
29060,Entech Controls,ROUND BAR OD50 ASTM A276 TYPE 431,NaN,ALPESH METALS,ROUND BAR OD50 ASTM A276 TYPE 431ALPESH METALS
29061,CESARE BONETTI INTERNATIONAL PVT. LTD.,"PTV 6"" #300 RF BVE WCB HW IBR",NaN,CESARE BONETTI INDIA PVT. LTD.,"PTV 6"" #300 RF BVE WCB HW IBRCESARE BONETTI IN..."
29062,R-Tech Products & Packaging Pvt Ltd,PP_MONOBOX_ST5,NaN,Shree Arun Packaging Co. Pvt. Ltd.,PP_MONOBOX_ST5Shree Arun Packaging Co. Pvt. Ltd.


In [9]:
## make all in lower case :

tranzact_data_combined['OEM'] = tranzact_data_combined['OEM'].map(lambda x: x if type(x)!=str else x.lower()) 
tranzact_data_combined['item'] = tranzact_data_combined['item'].map(lambda x: x if type(x)!=str else x.lower()) 
tranzact_data_combined['product'] = tranzact_data_combined['product'].map(lambda x: x if type(x)!=str else x.lower())

## Removing some punctuations:

punctuation = ['!', '$', '%', '&', '(', ')', '*', '+', ',', '.', ':', ';', '<', '=', '>', '?', '@', \
               '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '\t', '\n'] 
 

for i in punctuation:
    
    tranzact_data_combined['item']= tranzact_data_combined['item'].str.replace(i," ")
    
tranzact_data_combined['item'] = tranzact_data_combined['item'].str.replace(r'\s+',' ')

## 

## remove rows with any cell value = none

# tranzact_data_labelled_final = tranzact_data_labelled.dropna()
# tranzact_data_un_labelled_final = tranzact_data_un_labelled.dropna()

# tranzact_data_labelled_final = tranzact_data_labelled_final.reset_index(drop=True)
# tranzact_data_un_labelled_final = tranzact_data_un_labelled_final.reset_index(drop=True)

print(len(tranzact_data_combined))
tranzact_data_combined.tail()

29063


,OEM,item,product,supplier,find_duplicate
29058,r-tech products & packaging pvt ltd,polybag 10x15 inch pp,NaN,GLORY PACK INDUSTRIES,POLYBAG_10X15 INCH_PPGLORY PACK INDUSTRIES
29059,cesare bonetti india pvt. ltd.,"handle nut 1"" #600 htb a194 gr8",NaN,Shah Brothers,"HANDLE NUT 1"" #600,HTB,A194 Gr8Shah Brothers"
29060,entech controls,round bar od50 astm a276 type 431,NaN,ALPESH METALS,ROUND BAR OD50 ASTM A276 TYPE 431ALPESH METALS
29061,cesare bonetti international pvt. ltd.,"ptv 6"" #300 rf bve wcb hw ibr",NaN,CESARE BONETTI INDIA PVT. LTD.,"PTV 6"" #300 RF BVE WCB HW IBRCESARE BONETTI IN..."
29062,r-tech products & packaging pvt ltd,pp monobox st5,NaN,Shree Arun Packaging Co. Pvt. Ltd.,PP_MONOBOX_ST5Shree Arun Packaging Co. Pvt. Ltd.


In [10]:
from gensim.models import Word2Vec

from tqdm import tqdm

temp_corpus = tranzact_data_combined['item'].map(lambda x: x.split('.'))

corpus = []
for i in tqdm(range(len(temp_corpus))):
    for line in temp_corpus[i]:
        words = [x for x in line.split()]
        corpus.append(words)

100%|██████████| 29063/29063 [00:00<00:00, 42761.23it/s]


In [11]:
num_of_items = len(corpus)

num_of_words = 0
for line in corpus:
    num_of_words += len(line)
    
max_length = 0
for i in range(len(corpus)):
    max_length  = max(max_length, len(corpus[i]), len(corpus[i+1]))
    if i == len(corpus)-2:
        break
        
print('Num of sentences - %s'%(num_of_items))
print('Num of words - %s'%(num_of_words))
print('Max no. of words in a sentence - %s'%(max_length))

Num of sentences - 29063
Num of words - 206797
Max no. of words in a sentence - 103


In [12]:
## items to word2vec :

emb_dim = 80

model_1 = Word2Vec(corpus, size = emb_dim, window=5, \
               min_count=1, negative = 15, iter = 10, workers = 10, sg=1)

print(model_1)

Word2Vec(vocab=18013, size=80, alpha=0.025)


In [13]:
## Splitting the combined data into labelled data and unlabelled data :

# tranzact_data_unlabelled = tranzact_data_combined.loc[tranzact_data_combined['product'] == Null]

tranzact_data_labelled = tranzact_data_combined.loc[pd.notnull(tranzact_data_combined['product'])]

tranzact_data_un_labelled = tranzact_data_combined.drop(tranzact_data_labelled.index)

tranzact_data_labelled = tranzact_data_labelled.reset_index(drop=True)
tranzact_data_un_labelled = tranzact_data_un_labelled.reset_index(drop=True)

print(len(tranzact_data_labelled))
print(len(tranzact_data_un_labelled))
tranzact_data_un_labelled.tail()

11803
17260


,OEM,item,product,supplier,find_duplicate
17255,r-tech products & packaging pvt ltd,polybag 10x15 inch pp,NaN,GLORY PACK INDUSTRIES,POLYBAG_10X15 INCH_PPGLORY PACK INDUSTRIES
17256,cesare bonetti india pvt. ltd.,"handle nut 1"" #600 htb a194 gr8",NaN,Shah Brothers,"HANDLE NUT 1"" #600,HTB,A194 Gr8Shah Brothers"
17257,entech controls,round bar od50 astm a276 type 431,NaN,ALPESH METALS,ROUND BAR OD50 ASTM A276 TYPE 431ALPESH METALS
17258,cesare bonetti international pvt. ltd.,"ptv 6"" #300 rf bve wcb hw ibr",NaN,CESARE BONETTI INDIA PVT. LTD.,"PTV 6"" #300 RF BVE WCB HW IBRCESARE BONETTI IN..."
17259,r-tech products & packaging pvt ltd,pp monobox st5,NaN,Shree Arun Packaging Co. Pvt. Ltd.,PP_MONOBOX_ST5Shree Arun Packaging Co. Pvt. Ltd.


In [14]:
from collections import Counter
Counter(tranzact_data_labelled["product"])

Counter({'acid': 9,
         'actuator': 61,
         'adhesive': 12,
         'alloysteelbar': 63,
         'alloysteelcasting': 99,
         'alloysteelforging': 62,
         'aluminiumcasting': 3,
         'bearing': 128,
         'belt': 28,
         'bushbearing': 93,
         'carbonsteelbar': 125,
         'cibar': 5,
         'cicasting': 179,
         'cooling': 130,
         'cscasting': 500,
         'csforging': 104,
         'diaphragm': 28,
         'electrical': 1362,
         'electronic': 255,
         'fasteners': 1737,
         'gas': 21,
         'gasket': 241,
         'gearbox': 30,
         'glass': 22,
         'ic': 78,
         'inductor': 19,
         'lab-apparatus': 33,
         'machinery': 10,
         'manifold': 47,
         'measuringinstrumentelectrical': 6,
         'measuringinstrumentmechanical': 65,
         'mechanicalconnector': 289,
         'metalsheet': 358,
         'misc': 350,
         'motor': 38,
         'msbar': 20,
         'nameplate

In [15]:
## digitising 'product' and creating a dictionary for labelled data :

tranzact_data_labelled['product_id'] = tranzact_data_labelled['product'].factorize()[0]

product_id_df = tranzact_data_labelled[['product', 'product_id']].drop_duplicates().sort_values('product_id')
product_to_id = dict(product_id_df.values)
id_to_product = dict(product_id_df[['product_id', 'product']].values)

## removing puncuations from items of both labelled and un-labelled data :

# pd.options.mode.chained_assignment = None

# tranzact_data_labelled_final['item'] = tranzact_data_labelled_final['item'].str.replace(r'\W',' ')
# tranzact_data_un_labelled_final['item'] = tranzact_data_un_labelled_final['item'].str.replace(r'\W',' ')

## remove gaps between words to single gap

# tranzact_data_labelled_final['item'] = tranzact_data_labelled_final['item'].str.replace(r'\s+',' ')
# tranzact_data_un_labelled_final['item'] = tranzact_data_un_labelled_final['item'].str.replace(r'\s+',' ')

print(len(tranzact_data_labelled))
print(len(tranzact_data_un_labelled))
tranzact_data_labelled.tail()

11803
17260


,OEM,item,product,supplier,find_duplicate,product_id
11798,cesare bonetti india pvt. ltd.,"yoke casting 12"" #2500 gtv sa216 wcc qap",cscasting,AMI Alloys,"YOKE CASTING 12"" #2500 GTV, SA216 WCC, QAPAMI ...",17
11799,cesare bonetti india pvt. ltd.,"yoke cast 3"" #2500 gtv sa216 wcc qap",cscasting,Tulip Casting Pvt. Ltd.,"YOKE CAST, 3"" #2500 GTV, SA216 WCC, QAPTulip C...",17
11800,cesare bonetti india pvt. ltd.,"yoke casting 12"" #2500 gtv sa217 c12a qap",alloysteelcasting,AMI Alloys,"YOKE CASTING 12"" #2500 GTV, SA217 C12A, QAPAMI...",38
11801,cesare bonetti india pvt. ltd.,"yoke cast 10"" #150 tcv a217wc6",alloysteelcasting,RAJ ENGINEERS,"YOKE CAST 10"" #150 TCV; A217WC6RAJ ENGINEERS",38
11802,waaree industries pvt. ltd.,"yoke sleeve 8"" # 150 gtv a439 d2",cicasting,Waaree Industries Pvt.Ltd.,"YOKE SLEEVE;8"" # 150;GTV; A439 D2Waaree Indus...",16


In [16]:
tokenizer_obj_comb = Tokenizer(filters='!$%&()*+,.:;<=>?@[\\]^_`{|}~\t\n', split=' ' )
tokenizer_obj_lab = Tokenizer(filters='!$%&()*+,.:;<=>?@[\\]^_`{|}~\t\n', split=' ' )
tokenizer_obj_un_lab = Tokenizer(filters='!$%&()*+,.:;<=>?@[\\]^_`{|}~\t\n', split=' ' )

tokenizer_obj_comb.fit_on_texts(tranzact_data_combined['item'])
tokenizer_obj_lab.fit_on_texts(tranzact_data_labelled['item'])
tokenizer_obj_un_lab.fit_on_texts(tranzact_data_un_labelled['item'])

## pad sequences :

max_length_comb = max([len(s.split()) for s in tranzact_data_combined['item']])
max_length_lab = max([len(s.split()) for s in tranzact_data_labelled['item']])
max_length_un_lab = max([len(s.split()) for s in tranzact_data_un_labelled['item']])

# min_length = min([len(s.split()) for s in total_item_labelled_unlabelled['item']])

## define vocabulary size :

tokenizer_obj_lab_index = tokenizer_obj_lab.word_index

vocab_size_comb = len(tokenizer_obj_comb.word_index) + 1
vocab_size_lab = len(tokenizer_obj_lab.word_index) + 1
vocab_size_un_lab = len(tokenizer_obj_un_lab.word_index) + 1

# total_item_tokens = tokenizer_obj.texts_to_sequences(total_item_labelled_unlabelled['item'])
# total_item_tokens_1 = tokenizer_obj_1.texts_to_sequences(tranzact_data_labelled_final['item'])

# total_item_pad = pad_sequences(total_item_tokens, maxlen = max_length, padding = 'post')
# total_item_pad_1 = pad_sequences(total_item_tokens_1, maxlen = max_length, padding = 'post')

print(max_length_comb)
print(max_length_lab)
print(max_length_un_lab)
print('==============')
print(vocab_size_comb)
print(vocab_size_lab)
print(vocab_size_un_lab)

103
39
103
18014
9509
13067


In [17]:
temp_corpus_labelled = tranzact_data_labelled['item'].map(lambda x: x.split('.'))

corpus_labelled = []
for i in tqdm(range(len(temp_corpus_labelled))):
    for line in temp_corpus_labelled[i]:
        words_labelled = [x for x in line.split()]
        corpus_labelled.append(words_labelled)

100%|██████████| 11803/11803 [00:00<00:00, 44957.39it/s]


In [18]:
temp_corpus_un_labelled = tranzact_data_un_labelled['item'].map(lambda x: x.split('.'))

corpus_un_labelled = []
for i in tqdm(range(len(temp_corpus_un_labelled))):
    for line in temp_corpus_un_labelled[i]:
        words_un_labelled = [x for x in line.split()]
        corpus_un_labelled.append(words_un_labelled)

100%|██████████| 17260/17260 [00:00<00:00, 51719.20it/s]


In [19]:
num_of_items_labelled = len(corpus_labelled)

num_of_words_labelled = 0
for line in corpus_labelled:
    num_of_words_labelled += len(line)
    
max_length_labelled = 0
for i in range(len(corpus_labelled)):
    max_length_labelled  = max(max_length_labelled, len(corpus_labelled[i]), len(corpus_labelled[i+1]))
    if i == len(corpus_labelled)-2:
        break
        
        
num_of_items_un_labelled = len(corpus_un_labelled)

num_of_words_un_labelled = 0
for line in corpus_un_labelled:
    num_of_words_un_labelled += len(line)
    
max_length_un_labelled = 0
for i in range(len(corpus_un_labelled)):
    max_length_un_labelled  = max(max_length_un_labelled, len(corpus_un_labelled[i]), len(corpus_un_labelled[i+1]))
    if i == len(corpus_un_labelled)-2:
        break


print('Num of labelled sentences - %s'%(num_of_items_labelled))
print('Num of labelled words - %s'%(num_of_words_labelled))
print('Max no. of labelled words in a sentence - %s'%(max_length_labelled))
print('Num of un-labelled sentences - %s'%(num_of_items_un_labelled))
print('Num of un-labelled words - %s'%(num_of_words_un_labelled))
print('Max no. of un-labelled words in a sentence - %s'%(max_length_un_labelled))

Num of labelled sentences - 11803
Num of labelled words - 86127
Max no. of labelled words in a sentence - 39
Num of un-labelled sentences - 17260
Num of un-labelled words - 120670
Max no. of un-labelled words in a sentence - 103


In [160]:
## prepare embeddings

## Preparing embedding matrix

## nb_words = min(vocab_size, vocab_size_1)-1

embedding_matrix = np.zeros((vocab_size_lab, emb_dim))

for word, i in tokenizer_obj_lab_index.items():
    
    emb_vector = model_1.wv.word_vec(word)
    
    if emb_vector is not None:
        embedding_matrix[i] = emb_vector
        
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))


Null word embeddings: 1


In [21]:
print(type(embedding_matrix))
print(embedding_matrix.shape)
print(embedding_matrix[0])

<class 'numpy.ndarray'>
(9509, 80)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
## Total no. of unique words = vocab_size = 14707
## each word has got 150 parameters as per Embedding Dimension defined.
## Hence total Param # = 14707 * 150 = 2206050


In [ ]:
adam = adam(lr=0.1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [161]:
from keras.models import Sequential

model_2 = Sequential()
model_2.add(Embedding(vocab_size_lab, emb_dim, weights=[embedding_matrix],\
                    input_length=max_length_labelled, trainable=True))
model_2.add(SpatialDropout1D(0.2))
#model_2.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='tanh'))
#model_2.add(MaxPooling1D(pool_size=2))

model_2.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
#model_2.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model_2.add(Dense(256, activation='tanh'))
model_2.add(Dense(67, activation='softmax'))
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model_2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 39, 80)            760720    
_________________________________________________________________
spatial_dropout1d_7 (Spatial (None, 39, 80)            0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 128)               74240     
_________________________________________________________________
dense_9 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_10 (Dense)             (None, 67)                17219     
Total params: 885,203
Trainable params: 885,203
Non-trainable params: 0
_________________________________________________________________


In [152]:
## Creating X and Y variables for training. X = Items or item 'feature'. Y = 'product' or product category

X = []
for i in range(len(tranzact_data_labelled)):
    
    X.append(tranzact_data_labelled.iloc[i]['item'])
#Q = np.array(tranzact_data_3["product"])
Y = tranzact_data_labelled['product']
print(X[1])

print(Y[1])


accumulator - model a-as 5126 3/4" connection - emerson make
cooling


In [153]:
X_tokens = tokenizer_obj_lab.texts_to_sequences(X)
#X_test_tokens = tokenizer_obj_lab.texts_to_sequences(X_test)

X_pad = pad_sequences(X_tokens, maxlen = max_length_labelled, padding = 'post')
#X_test_pad = pad_sequences(X_test_tokens, maxlen = max_length_labelled, padding = 'post')

print(X_tokens[0])

[1454, 237, 2814, 792]


In [154]:
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer()

Y_coded = encoder.fit_transform(Y)

print(Y_coded.shape)



(11803, 67)


In [155]:
#train test split in 80% / 20% ratio

from sklearn.model_selection import train_test_split
X_train_pad, X_test_pad, Y_train_coded, Y_test_coded = train_test_split(X_pad, Y_coded, test_size=0.20, random_state = 5)


In [ ]:
# total_item_tokens = tokenizer_obj.texts_to_sequences(total_item_labelled_unlabelled['item'])
# total_item_tokens_1 = tokenizer_obj_1.texts_to_sequences(tranzact_data_labelled_final['item'])

# total_item_pad = pad_sequences(total_item_tokens, maxlen = max_length, padding = 'post')
# total_item_pad_1 = pad_sequences(total_item_tokens_1, maxlen = max_length, padding = 'post')


In [48]:
## X_train and X_test word embedding :

#X_train_tokens = tokenizer_obj_lab.texts_to_sequences(X_train)
#X_test_tokens = tokenizer_obj_lab.texts_to_sequences(X_test)

#X_train_pad = pad_sequences(X_train_tokens, maxlen = max_length_labelled, padding = 'post')
#X_test_pad = pad_sequences(X_test_tokens, maxlen = max_length_labelled, padding = 'post')

In [156]:
print(X_train_pad.shape)
print(X_test_pad.shape)
print(max_length_lab)
print(vocab_size_lab)

print(Y_train_coded.shape)

(9442, 39)
(2361, 39)
39
9509
(9442, 67)


In [157]:
X_test_pad[0]

array([6873,  162, 1575, 1428,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [158]:
print(Y_test_coded[0])
print(Y_test_coded[0].shape)
print(Y_test_coded.shape)
print(([Y_test_coded][0]).shape)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
(67,)
(2361, 67)
(2361, 67)


In [71]:
encoder.inverse_transform(Y_test_coded)

array(['tool', 'metalsheet', 'valve', ..., 'spring', 'pipe', 'pipe'],
      dtype='<U29')

In [88]:

ppp = [encoder.inverse_transform([Y_test_coded][0])][0][0]
print(ppp)

tool


In [162]:
model_2.fit(X_train_pad,Y_train_coded,batch_size=64,epochs=36,
          validation_split=0.2)

Train on 7553 samples, validate on 1889 samples
Epoch 1/36
7553/7553 [==============================] - 15s 2ms/step - loss: 2.4471 - acc: 0.4229 - val_loss: 1.3333 - val_acc: 0.6496
Epoch 2/36
7553/7553 [==============================] - 14s 2ms/step - loss: 1.3037 - acc: 0.6567 - val_loss: 0.9006 - val_acc: 0.7485
Epoch 3/36
7553/7553 [==============================] - 14s 2ms/step - loss: 0.9356 - acc: 0.7470 - val_loss: 0.6884 - val_acc: 0.8163
Epoch 4/36
7553/7553 [==============================] - 13s 2ms/step - loss: 0.7435 - acc: 0.7977 - val_loss: 0.5821 - val_acc: 0.8465
Epoch 5/36
7553/7553 [==============================] - 13s 2ms/step - loss: 0.5762 - acc: 0.8424 - val_loss: 0.5023 - val_acc: 0.8602
Epoch 6/36
7553/7553 [==============================] - 13s 2ms/step - loss: 0.4692 - acc: 0.8740 - val_loss: 0.4387 - val_acc: 0.8862
Epoch 7/36
7553/7553 [==============================] - 14s 2ms/step - loss: 0.3884 - acc: 0.8987 - val_loss: 0.4193 - val_acc: 0.8894
Epoch 8

In [163]:
#test_sequences = tok.texts_to_sequences(X_test)
#test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
accr = model_2.evaluate(X_test_pad,Y_test_coded)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

2361/2361 [==============================] - 1s 473us/step
Test set
  Loss: 0.363
  Accuracy: 0.931


In [108]:
#pred_iction = model_2.predict(X_test_pad[0])
print(X_test_pad[0].shape)
print(X_test_pad.shape)
aaa = X_test_pad[0].reshape(39,1)
print(aaa.shape)

(39,)
(2361, 39)
(39, 1)


In [130]:
print([X_test_pad][0])

[[6873  162 1575 ...    0    0    0]
 [ 206    5   50 ...    0    0    0]
 [ 597   46   82 ...    0    0    0]
 ...
 [   6   92  159 ...    0    0    0]
 [  20    9  136 ...    0    0    0]
 [  68 1409  699 ...    0    0    0]]


In [125]:
print(X_test_pad[1])

[ 206    5   50 7746  840    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]


In [134]:
pr = [model_2.predict([X_test_pad][0])][0][0]
aaa = [model_2.predict([X_test_pad][0])][0][1]
print(aaa)

[2.43695065e-13 5.37912215e-11 6.06054513e-14 1.58411409e-11
 3.62898242e-13 3.07999654e-12 1.80659944e-13 7.98073155e-14
 2.48591037e-10 8.95355734e-10 5.61677198e-11 2.49392304e-13
 5.67071000e-12 9.73222963e-16 9.79940296e-11 9.85307036e-09
 7.43214934e-11 6.00585448e-09 2.84825691e-10 1.18157439e-09
 3.32950654e-13 8.60682375e-14 6.53146735e-16 4.27782375e-12
 8.61340166e-12 1.55950621e-12 5.95001002e-12 2.48350971e-12
 7.43745939e-13 3.18562612e-15 1.34125052e-13 5.68245483e-13
 9.99999404e-01 3.40822481e-08 4.58213155e-12 1.57077355e-11
 2.80499108e-07 2.46402183e-13 6.62316957e-09 3.17122935e-14
 5.72866199e-09 1.58481908e-10 2.00847380e-16 3.10599688e-07
 1.40421647e-10 7.88168975e-11 2.69482922e-14 1.05637173e-10
 1.26338556e-14 2.54071585e-13 4.12701096e-13 3.03979135e-08
 3.66830812e-17 4.13453669e-16 5.29142456e-15 1.92841840e-10
 2.86646087e-13 6.45367426e-16 1.17023458e-09 5.59293237e-11
 5.30485248e-12 7.18299443e-13 2.55449429e-11 2.64648465e-14
 1.26343805e-11 7.793420

In [91]:
print(type(X_test_pad[0]))
print(type([X_test_pad][0]))
print('=====================')
print((X_test_pad[0]).shape)
print(([X_test_pad][0]).shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(39,)
(2361, 39)


In [135]:
print(pr.shape)
print(pr)
ppr = pr.reshape(1,67)
paaa = aaa.reshape(1,67)

(67,)
[6.4785499e-06 6.2430248e-11 5.0905498e-08 1.4635412e-12 2.6252878e-12
 1.0080431e-11 5.6519306e-10 1.4289529e-06 6.9042760e-10 1.7831733e-10
 2.1181404e-10 1.2298527e-14 1.6803699e-12 1.5196718e-09 9.9246371e-09
 5.9822971e-12 8.0213539e-08 4.2344413e-06 7.4888703e-08 2.1124679e-06
 1.0061122e-12 2.1798749e-09 1.2109756e-13 5.4485174e-14 1.6755372e-12
 1.8148221e-11 7.0184125e-09 4.6026533e-10 6.5030742e-10 2.3398725e-10
 1.1460697e-09 3.3852853e-12 4.0140020e-09 1.0219968e-05 3.0548208e-06
 1.3218934e-13 4.5816320e-10 5.3340515e-10 5.3676023e-16 5.6870773e-13
 8.1808907e-11 5.9328343e-08 1.4899115e-06 5.6519839e-14 5.8173253e-09
 3.7106645e-07 4.4050662e-12 8.2427898e-11 8.2608983e-07 2.7882590e-11
 1.1674641e-09 2.3090813e-14 1.1537603e-11 1.0172010e-09 1.7543336e-13
 3.6769111e-14 3.6106526e-10 6.2531993e-12 5.3707188e-11 7.2918711e-13
 3.3740302e-12 9.9996948e-01 2.1328434e-11 5.9378920e-09 7.5311508e-09
 1.7534629e-14 8.5223384e-11]


In [136]:
#ppp = [encoder.inverse_transform([Y_test_coded][0])][0][0]

ppp = encoder.inverse_transform([ppr][0])[0]
aaap = encoder.inverse_transform([paaa][0])[0]

In [137]:
print(ppp)
print(aaap)

tool
metalsheet


In [ ]:
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))


In [225]:
text = ['solenoid coil']

In [226]:
text_tokens = tokenizer_obj_lab.texts_to_sequences(text)
#X_test_tokens = tokenizer_obj_lab.texts_to_sequences(X_test)

text_pad = pad_sequences(text_tokens, maxlen = max_length_labelled, padding = 'post')
#X_test_pad = pad_sequences(X_test_tokens, maxlen = max_length_labelled, padding = 'post')


In [227]:
b = [model_2.predict([text_pad][0])][0][0]

In [228]:
bb = b.reshape(1,67)

In [229]:
pred = encoder.inverse_transform([bb][0])[0]

In [230]:
print(pred)

cooling
